In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes --quiet


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="motivabot_dataset.jsonl", split="train")
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Preview a few samples
dataset["train"].select(range(5))


Dataset({
    features: ['input', 'output'],
    num_rows: 5
})

In [ ]:
def preprocess(example):
    example["input_text"] = "Motivate: " + example["input"]
    example["target_text"] = example["output"]
    return example

dataset = dataset.map(preprocess)


In [ ]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
max_input_length = 128
max_target_length = 64

def tokenize(example):
    model_input = tokenizer(
        example["input_text"],
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        example["target_text"],
        max_length=max_target_length,
        truncation=True,
        padding="max_length"
    )
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./motivabot-model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=3e-4,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    report_to="none"
)


In [ ]:

import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import DataCollatorForSeq2Seq, Trainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


<ipython-input-19-19bc3cdee798>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


TrainOutput(global_step=125, training_loss=3.510943799972534, metrics={'train_runtime': 69.4514, 'train_samples_per_second': 14.399, 'train_steps_per_second': 1.8, 'total_flos': 46472626176000.0, 'train_loss': 3.510943799972534, 'epoch': 5.0})

In [ ]:
import torch

# Ensure model and inputs are on the same device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

input_text = "Motivate: I feel like a failure"
inputs = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    temperature=0.9,
    num_beams=4,
    early_stopping=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Failure is the only way to do good work.


In [ ]:
!pip install gradio --quiet
import gradio as gr
import torch

# Ensure model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def motivate_with_bot(user_input):
    try:
        prompt = "Motivate: " + user_input
        inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

        outputs = model.generate(
            inputs,
            max_new_tokens=50,
            do_sample=True,
            top_k=50,
            temperature=0.9,
            num_beams=4,
            early_stopping=True
        )

        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# Define Gradio interface
interface = gr.Interface(
    fn=motivate_with_bot,
    inputs=gr.Textbox(lines=2, placeholder="How are you feeling today?"),
    outputs="text",
    title="💬 MotivaBot: Your Personal Motivation Coach",
    description="Enter your current mood or challenge, and let MotivaBot cheer you on with a quote!"
)

# Launch the app
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a86eda20255e002a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
